In [2]:
%load_ext jupyter_black

In [1]:
import json
from glob import glob
from pathlib import Path

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
# ml

import tensorflow
from sklearn.manifold import LocallyLinearEmbedding
# plotting
import matplotlib.pyplot as plt



all_files = sorted(glob("/workspaces/sppp/data/*"))

2022-07-19 14:00:33.723891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 14:00:33.723933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:

def open_file(filepath: Path):
    with filepath.open("rb") as fin:
        return json.load(fin)

def pre():
    ...


moisture_indicies = [
    "EBSHEAR",
    "SRH01KM",
    "MEANWIND_1-3kmAGL",
    "MESH",
    "VIL_DENSITY",
    "FLASH_RATE",
    "FLASH_DENSITY",
    "MAXLLAZ",
    "P98LLAZ",
    "P98MLAZ",
    "MAXRC_EMISS",
    "MAXRC_ICECF",
    "WETBULB_0C_HGT",
    "PWAT",
    "CAPE_M10M30",
    "LJA",
    "SIZE",
    "AVG_BEAM_HGT",
    "MOTION_EAST",
    "MOTION_SOUTH",
    "PS",
    "ID",
    "centroid",
]
stablitity_indicies = [
    "MUCAPE",
    "MLCAPE",
    "MLCIN",
]
lift_indicies = []
growth_indicies = []
# from sklearn
# from tensorflow
lle = LocallyLinearEmbedding(n_components=3, n_neighbors=2)
from typing import NewType
FeatureCollection = NewType("FeatureCollection",dict)
def to_dataframe(fc:FeatureCollection)->pd.DataFrame:
    df = gpd.GeoDataFrame.from_features(fc["features"])
    df["validTime"] = pd.to_datetime(fc["validTime"], format="%Y%m%d_%H%M%S %Z")
    df["centroid"] = df["geometry"].centroid
    stab = lle.fit_transform(df[stablitity_indicies])
    
    return df.set_index(["validTime", "ID"])


def to_midf()->pd.DataFrame:

    def generate():
        for file in all_files:
            fc = open_file(Path(file))
            yield to_dataframe(fc)
    return pd.concat(generate())

to_midf()



,geometry,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,...,CAPE_M10M30,LJA,SIZE,AVG_BEAM_HGT,MOTION_EAST,MOTION_SOUTH,PS,ID,validTime,centroid
0,"POLYGON ((-96.06000 38.40000, -95.99000 38.400...",485,0,0,48.6,63,12.9,0.25,1.67,10,...,129,0.0,515,3.20 kft / 0.98 km,15.042,-14.019,11,89234,2021-10-11 00:00:53+00:00,POINT (-96.06829 38.23883)
1,"POLYGON ((-80.66000 24.76000, -80.62000 24.760...",2691,2179,-2,22.9,14,4.0,0.43,1.60,7,...,668,0.0,45,4.03 kft / 1.23 km,0.423,-2.061,4,89321,2021-10-11 00:00:53+00:00,POINT (-80.63205 24.72810)
2,"POLYGON ((-96.98000 36.80000, -96.87000 36.800...",1367,937,-19,61.9,148,17.6,1.43,2.85,60,...,319,1.2,355,5.01 kft / 1.53 km,9.61,-11.265,98,89467,2021-10-11 00:00:53+00:00,POINT (-96.94631 36.71592)
3,"POLYGON ((-75.15000 35.44000, -75.11000 35.440...",2493,2076,0,24.0,19,16.3,0.05,0.62,0,...,420,0.0,50,11.92 kft / 3.63 km,-1.606,0.677,2,89470,2021-10-11 00:00:53+00:00,POINT (-75.12905 35.39801)
4,"POLYGON ((-98.29000 35.59000, -98.27000 35.590...",2035,1656,-17,74.6,197,26.4,2.19,3.29,181,...,603,1.2,1685,3.96 kft / 1.21 km,12.789,-8.273,100,89519,2021-10-11 00:00:53+00:00,POINT (-98.23654 35.12736)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"POLYGON ((-97.10000 37.72000, -97.05000 37.720...",576,0,0,52.1,74,8.4,0.00,0.52,0,...,160,0.0,124,1.04 kft / 0.32 km,11.984,-13.031,4,90389,2021-10-11 01:00:58+00:00,POINT (-97.09923 37.65983)
83,"POLYGON ((-98.03000 34.15000, -97.99000 34.100...",2113,2067,-31,59.3,192,36.4,0.25,0.88,4,...,704,0.0,48,4.56 kft / 1.39 km,13.982,-3.008,33,90390,2021-10-11 01:00:58+00:00,POINT (-98.02261 34.09865)
84,"POLYGON ((-98.87000 31.84000, -98.85000 31.830...",2362,1910,-22,67.5,197,32.8,0.04,0.77,0,...,635,0.0,91,3.57 kft / 1.09 km,17.65,-6.134,10,90391,2021-10-11 01:00:58+00:00,POINT (-98.89423 31.78376)
85,"POLYGON ((-99.03000 31.68000, -99.01000 31.680...",2208,1744,-15,66.8,170,32.2,0.59,1.75,12,...,594,0.0,210,4.81 kft / 1.47 km,13.725,-0.784,64,90392,2021-10-11 01:00:58+00:00,POINT (-99.11583 31.53175)
